# Temporal response functions

Temporal response functions (TRFs) are an alternative approach to understanding the relationship between stimulus events and M/EEG activity. The idea is to estimate a general linear model, with each row of the design matrix capturing a temporally-lagged version of the regressor of interest (Crosse 2016, Ehinger 2019). This can be used for discrete events (as in the current study), but can also be used for continuous, time-evolving regressors. 

A key advantage of this approach is that it allows for 'unmixing' of temporally-overlapping neural responses to an event (Smith 2015). On the other hand, if events are sufficiently temporally separated and discrete, then it is mathematically equivalent to event related averaging. The results from this approach should therefore be equivalent to those identifed in the EventRelatedFields notebook.

## Preparation

Import the relevant modules. We plan to use Julia+Unfold. Advice on how to install this is available at https://unfoldtoolbox.github.io/Unfold.jl/dev/generated/HowTo/juliacall_unfold/. We can also try mTRF, see https://mtrfpy.readthedocs.io/en/latest/index.html 

In [1]:
import os.path as op
import os

import pandas as pd
import numpy as np

#from mne_bids import BIDSPath, read_raw_bids # If you don't have mne_bids installed, comment this line.
# Import the Julia package manager
from juliacall import Pkg as jlPkg
# Activate the environment in the current folder
jlPkg.activate(".")
# Install Unfold (in the activated environment)
jlPkg.add("Unfold")

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Note that currently, importing mtrf, MNE and mne_bids seems to hang when Julia has been activated :-(

In [2]:
# import mne

# from mne_bids import BIDSPath, read_raw_bids # If you don't have mne_bids installed, comment this line.

# import mtrf as mtf

In [ ]:
mtf.model.load_sample_data() # Load sample data 

### Loading the data - if you have mne_bids installed

In [ ]:
subject = '01'
session = '01'
task = 'SpAtt'
run = '01'
meg_suffix = 'meg'
ica_suffix = 'ica'
ann_suffix = 'ann'
epo_suffix = 'epo'


preproc_root = r'/Users/lhunt/scratch/projects/FLUX_pipeline/derivatives/Preprocessing'
analysis_root = r'/Users/lhunt/scratch/projects/FLUX_pipeline/derivatives/Analysis'

bids_path_preproc = BIDSPath(subject=subject, session=session,
            task=task, run=run, suffix=ica_suffix, datatype='meg',
            root=preproc_root, extension='.fif', check=False)

ann_fname = bids_path_preproc.basename.replace(ica_suffix, ann_suffix) # output filename
ann_fname = ann_fname.replace('fif', 'csv') # output filename

ann_fname_1 = op.join(bids_path_preproc.directory, ann_fname)
ann_fname_2 = ann_fname_1.replace('run-01', 'run-02')

Read the pre-processed data, after ICA. Note that we are *not* loading in the epoched data here, as we want to work with the continuous data instead.

In [ ]:
# This section only runs on run=01 for illustration purpose 
bids_path_preproc.update(run='01')
raw = read_raw_bids(bids_path=bids_path_preproc, 
            extra_params={'preload':False},
            verbose=True)


Now, read the events from the stimulus channel (STI101), and also read the artefacts that were detected in the ArtefactAnnotation script from the CSV file.

In [ ]:
# Reading the events from the  raw file
events, events_id = mne.events_from_annotations(raw, event_id='auto')

# Reading the artifact rejection annotations
ann1 = mne.read_annotations(ann_fname_1)
print(ann1)
raw.set_annotations(ann1)

In [ ]:
%matplotlib inline
plt.stem(events[:,0][:80], events[:,2][:80])
plt.xlabel('samples')
plt.ylabel('Trigger value (STI101)')
plt.show()

# References

Crosse MJ, Di Liberto GM, Bednar A and Lalor EC (2016) The Multivariate Temporal Response Function (mTRF) Toolbox: A MATLAB Toolbox for Relating Neural Signals to Continuous Stimuli. Front. Hum. Neurosci. 10:604. doi: 10.3389/fnhum.2016.00604

Ehinger BV, Dimigen O. 2019. Unfold: an integrated toolbox for overlap correction, non-linear modeling, and regression-based EEG analysis. PeerJ 7:e7838 https://doi.org/10.7717/peerj.7838

Smith NJ, Kutas M. Regression-based estimation of ERP waveforms: I. The rERP framework. Psychophysiology. 2015 Feb;52(2):157-68. doi: 10.1111/psyp.12317. Epub 2014 Aug 21. PMID: 25141770; PMCID: PMC5308234.
